In [159]:
# This is Lecture05 - Exercise 2 of the "Data Science" class 
# at Technische Hochschule Rosenheim

# Iris Dataset revisited

# Part II - Outliers and Merging

Let's continue working on the three 'xxx_nmv' DataFrames. 

### Exercise II.1

Load the DataFrames from the previous part. A typical next step is to look at the distribution of each individual attribute in each of these three DataFrames. We will consider all values which are further than 3-times the standard-deviation from the mean to be outliers ("z-score outliers"). Which values in which DataFrame are outliers? Create three new DataFrames 'xxx_clean' where the tuples with outliers are removed, based on the 'xxx_nmv' DataFrames.

In [160]:
%matplotlib notebook
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [161]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [162]:
## ---------- SOLUTIONS
setosa_nmv = pd.read_csv("output/setosa_nmv.csv", index_col=0)
versicolor_nmv = pd.read_csv("output/versicolor_nmv.csv", index_col=0)
virginica_nmv = pd.read_csv("output/virginica_nmv.csv", index_col=0)

display("setosa_nmv.head()", "versicolor_nmv.head()", "virginica_nmv.head()")

setosa_nmv.head()
   index  sepal length  sepal width  petal length  petal width
0      0           5.1          3.5           1.4      0.20000
1      1           4.9          3.0           1.4      0.20000
2      2           4.7          3.2           1.3      0.20000
3      3           4.9          3.0           1.4      0.25614
4      4           4.6          3.1           1.5      0.20000

versicolor_nmv.head()
   sepal length  sepal width  petal length  petal width
0           7.0          3.2          0.47         0.14
1           6.4          3.2          0.45         0.15
2           6.9          3.1          0.49         0.15
3           5.5          2.3          0.40         0.13
4           6.5          2.8          0.46         0.15

virginica_nmv.head()
   sepal length  sepal width  petal length  petal width
#                                                      
1           6.3          3.3          0.60         0.25
2           5.8          2.7          0.51         0.19
3           7.1          3.0          0.59         0.21
4           6.3          2.9          0.56         0.18
5           6.5          3.0          0.58         0.22

In [163]:
display("setosa_nmv.describe().T", "versicolor_nmv.describe().T", "virginica_nmv.describe().T")

,count,mean,std,min,25%,50%,75%,max
index,60.0,29.500000,17.464249,0.0,14.75,29.5,44.250,59.0
sepal length,60.0,5.711864,5.562986,4.3,4.80,5.0,5.200,48.0
sepal width,60.0,4.394828,5.092814,2.3,3.20,3.4,3.725,32.0
petal length,60.0,2.012281,2.838302,1.0,1.40,1.5,1.600,19.0
petal width,60.0,0.256140,0.141442,0.1,0.20,0.2,0.300,1.0
,count,mean,std,min,25%,50%,75%,max
sepal length,50.0,5.9360,0.516171,4.9,5.600,5.900,6.30,7.00
sepal width,50.0,2.7700,0.313798,2.0,2.525,2.800,3.00,3.40
petal length,50.0,0.4260,0.046991,0.3,0.400,0.435,0.46,0.51
petal width,50.0,0.1326,0.019775,0.1,0.120,0.130,0.15,0.18


In [164]:
display('setosa_nmv.describe().T[["mean", "std"]]', 'versicolor_nmv.describe().T[["mean", "std"]]', 'viginica_nmv.describe().T[["mean", "std"]]')

,mean,std
index,29.500000,17.464249
sepal length,5.711864,5.562986
sepal width,4.394828,5.092814
petal length,2.012281,2.838302
petal width,0.256140,0.141442
,mean,std
sepal length,5.9360,0.516171
sepal width,2.7700,0.313798
petal length,0.4260,0.046991
petal width,0.1326,0.019775


In [165]:
clean = []
flowers = ["setosa_nmv", "versicolor_nmv", "virginica_nmv"]
for c in flowers:    
    dataframe = eval(c)
    means = dataframe.mean()
    stds = dataframe.std()
    outliers = dataframe[((dataframe < means + 3 * stds) | (dataframe > means - 3 * stds))]
    outliers.dropna(how="any", inplace=True)
    outliers["class"] = c
    clean.append(outliers)
    # print(outliers)
clean

[    index  sepal length  sepal width  petal length  petal width       class
 0       0      5.100000     3.500000      1.400000      0.20000  setosa_nmv
 1       1      4.900000     3.000000      1.400000      0.20000  setosa_nmv
 2       2      4.700000     3.200000      1.300000      0.20000  setosa_nmv
 3       3      4.900000     3.000000      1.400000      0.25614  setosa_nmv
 4       4      4.600000     3.100000      1.500000      0.20000  setosa_nmv
 5       5      5.000000     3.600000      1.400000      0.20000  setosa_nmv
 6       6      5.400000     3.900000      1.700000      0.40000  setosa_nmv
 7       7      4.600000     3.400000      1.400000      0.30000  setosa_nmv
 8       8      5.000000     3.400000      1.500000      0.20000  setosa_nmv
 9       9      4.600000     3.500000      2.012281      0.10000  setosa_nmv
 10     10      4.400000     2.900000      1.400000      0.20000  setosa_nmv
 11     11      4.900000     3.100000      1.500000      0.10000  setosa_nmv

### Exercise II.2

Combine the three 'xxx_clean' DataFrames into one DataFrame 'iris' by adding a new column 'class' which is set to 'Iris-setosa', 'Iris-versicolor' or 'Iris-virginica' respectively. Make sure the index in 'iris' are unique numbers from 0 to however-many-tuples-you-have. Save your result in a file 'output/iris_clean.csv'.


In [166]:
## ---------- SOLUTIONS
iris = clean[0].append(clean[1]).append(clean[2])
iris.reset_index(inplace=True)
del iris["index"]
iris.describe()

C:\Users\Felix\AppData\Local\Temp\ipykernel_11164\3311844987.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  iris = clean[0].append(clean[1]).append(clean[2])
C:\Users\Felix\AppData\Local\Temp\ipykernel_11164\3311844987.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  iris = clean[0].append(clean[1]).append(clean[2])


,level_0,sepal length,sepal width,petal length,petal width
count,160.000000,160.000000,160.000000,160.000000,160.000000
mean,26.687500,6.055699,3.443060,1.061230,0.200803
std,15.782399,3.439236,3.200032,1.881402,0.101960
min,0.000000,4.300000,2.000000,0.300000,0.100000
25%,13.000000,5.100000,2.800000,0.467500,0.140000
50%,26.500000,5.700000,3.000000,0.560000,0.200000
75%,40.000000,6.400000,3.400000,1.400000,0.210000
max,59.000000,48.000000,32.000000,19.000000,1.000000


------